# Default notebook

This default notebook is executed using Databricks Workflows as defined in resources/delta_live_store_job.yml.

In [ ]:
%pip install -qr ../requirements-dev.txt 
%pip install -q --upgrade databricks-sdk
dbutils.library.restartPython()

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dbrx.dls.store import DeltaLiveStore, DeltaLiveEntity
from dbrx.dls.filters import by_is_enabled, by_is_latest, Predicate
from dbrx.dls.loader import DeltaLiveStoreLoader, YamlLoader

In [ ]:
dbutils.widgets.text("configPath", "")
dbutils.widgets.text("deltaLiveStoreTable", "")

In [ ]:
config_path: str = dbutils.widgets.get("configPath")
delta_live_store_table: str = dbutils.widgets.get("deltaLiveStoreTable")

config_path = config_path or None
delta_live_store_table = delta_live_store_table or None

print(f"config_path: {config_path}")
print(f"delta_live_store_table: {delta_live_store_table}")

if config_path is None:
    raise ValueError("config_path is not set")
if delta_live_store_table is None:
    raise ValueError("delta_live_store_table is not set")


In [ ]:
store: DeltaLiveStore = DeltaLiveStore(delta_live_store_table)
store.initialize()
loader: DeltaLiveStoreLoader = YamlLoader(config_path)
loader.load(store)

In [ ]:

predicate: Predicate = by_is_latest() & by_is_enabled()
display(store.find_entities(predicate).to_df())